# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_data = pd.read_csv("../output/city_data.csv")
city_data.head()

,City,Latitude,Longitude,Maximum Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
0,Rikitea,-23.12,-134.97,72.48,77,100,16.75,PF,1601736544
1,Avarua,-21.21,-159.78,71.60,83,32,3.36,CK,1601736545
2,Sitka,57.05,-135.33,51.80,87,40,6.93,US,1601736546
3,Port Alfred,-33.59,26.89,62.17,82,91,22.21,ZA,1601736546
4,Mataura,-46.19,168.86,55.99,89,100,11.99,NZ,1601736547


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Access maps with unique API key
gmaps.configure(g_key)

# Store latitude and longitude in Locations
locations = city_data[["Latitude", "Longitude"]]

# Fill NaN values and convert to float
humidity = city_data["Humidity"].astype(float)

# Plot heatmap
fig = gmaps.figure()

# Add heatmap layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100, point_radius=5)
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
# Narrow down the cities to fit weather conditions
ideal_weather_df = city_data.loc[(city_data["Maximum Temperature"] > 70) &
                         (city_data["Maximum Temperature"] < 80) &
                         (city_data["Wind Speed"] < 10) &
                         (city_data["Cloudiness"] == 0)]
                        
ideal_weather_df

,City,Latitude,Longitude,Maximum Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
28,Guerrero Negro,27.98,-114.06,77.41,55,0,4.79,MX,1601736564
82,Talwāra,31.95,75.87,75.85,38,0,2.44,IN,1601736605
104,Boyuibe,-20.42,-63.28,78.80,44,0,5.82,BO,1601736621
466,San Quintín,30.48,-115.95,77.31,47,0,1.79,MX,1601736888
481,Almeria,37.17,-2.33,71.01,59,0,6.93,ES,1601736898


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
# Store into variable named hotel_df
hotel_df = ideal_weather_df.copy()

# Add a "Hotel Name" column to the DataFrame
hotel_df["Hotel Name"] = ""

# Set parameters to search for hotels with 5000 meters
params = {"key": g_key,
          "radius": 5000,
          "keyword": "hotel"
         }

# Using Google Places API to find the first hotel for each city located within 5000 meters of each city coordinates.
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():
    
    # get coordinates from df
    lat = row["Latitude"] 
    lng = row["Longitude"]

    # Add coordinates to params dict
    params['location'] = (f"{lat}, {lng}")
    
    # Make API request
    response = requests.get(base_url, params=params).json()
    
    # Extract hotel name
    # Since some data may be missing, we incorporate a try-except to skip any that are missing
    try:
        results = response['results'][0]['name']
        
        # Store the first hotel result into the DataFrame
        hotel_df.loc[index, 'Hotel Name'] = results
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

hotel_df

Missing field/result... skipping.


,City,Latitude,Longitude,Maximum Temperature,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
28,Guerrero Negro,27.98,-114.06,77.41,55,0,4.79,MX,1601736564,Hotel TerraSal
82,Talwāra,31.95,75.87,75.85,38,0,2.44,IN,1601736605,kesri hotel and restaurant
104,Boyuibe,-20.42,-63.28,78.80,44,0,5.82,BO,1601736621,Hotel oandraY
466,San Quintín,30.48,-115.95,77.31,47,0,1.79,MX,1601736888,Hotel Jardines Baja
481,Almeria,37.17,-2.33,71.01,59,0,6.93,ES,1601736898,


In [6]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [7]:
# Add a hotel marker layer ontop of humidity heatmap with each pin containing the **Hotel Name**, **City**, and **Country**
hotel_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

# Create combined map
fig = gmaps.figure()

fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)

fig

Figure(layout=FigureLayout(height='420px'))